- 가시성 확인 규칙은 튜플의 t_xmin 및 t_xmax, clog 및 획득한 트랜잭션 스냅샷을 모두 사용하여 각 튜플이 표시되는지 여부를 결정하는 데 사용되는 규칙 집합.

- 하위 트랜잭션과 관련된 규칙을 생략하고 t_ctid에 대한 논의를 무시.
  - 즉 트랜잭션 내에서 두 번 이상 업데이트된 튜플을 고려하지 않음.

- 선정된 규칙의 개수는 10개이며, 세 가지 경우로 분류할 수 가능(더 복잡하게 존재).

**Status of t_xmin is ABORTED**

- t_xmin 상태가 ABORTED인 튜플은 이 튜플을 삽입한 트랜잭션이 중단되었기 때문에 항상 표시되지 않음 (규칙 1).



```
/* t_xmin status == ABORTED */
Rule 1:	  IF t_xmin status is 'ABORTED' THEN
                 RETURN 'Invisible'
          END IF
```

- 이 규칙은 다음과 같은 수학적 표현으로 명시적으로 표현.

**규칙 1:**
- Status(t_xmin) = ABORTED인 경우 ⇒ Invisible

**Status of t_xmin is IN_PROGRESS**

- t_xmin 상태가 IN_PROGRESS인 튜플은 한 가지 조건을 제외하고는 본질적으로 보이지 않음 (규칙 3 및 4).



```
 /* t_xmin status == IN_PROGRESS */
       IF t_xmin status is 'IN_PROGRESS' THEN
              IF t_xmin = current_txid THEN
Rule 2:              IF t_xmax = INVALID THEN
                           RETURN 'Visible'
Rule 3:              ELSE  /* this tuple has been deleted or updated  */
                           /* by the current transaction itself.      */
                            RETURN 'Invisible'
                     END IF
Rule 4:       ELSE   /* t_xmin &ne; current_txid */
                     RETURN 'Invisible'
              END IF
       END IF
```

- 이 튜플이 다른 트랜잭션에 의해 삽입되고 t_xmin의 상태가 IN_PROGRESS인 경우 이 튜플은 분명히 보이지 않음 (규칙 4).

- t_xmin이 현재 txid와 같고(즉, 이 튜플은 현재 트랜잭션에 의해 삽입됨) t_xmax가 INVALID 가 아닌 경우 이 튜플은 현재 트랜잭션에 의해 업데이트되거나 삭제되었기 때문에 표시되지 않음 (규칙 3).

- 예외 조건은 이 튜플이 현재 트랜잭션에 의해 삽입되고 t_xmax가 INVALID인 경우.
  - 이 경우 이 튜플은 현재 트랜잭션 자체에 의해 삽입된 튜플이기 때문에 현재 트랜잭션에서 표시 되어야 함(규칙 2).

**규칙 2:**
- 상태(t_xmin) = IN_PROGRESS인 경우∧t_xmin = current_txid ∧t_xmax = INVAILD ⇒Visible

**규칙 3:**
- 상태(t_xmin) = IN_PROGRESS인 경우∧ t_xmin = current_txid ∧ t_xmax≠ INVAILD  Invisible

**규칙 4:**
- 상태(t_xmin) = IN_PROGRESS인 경우 ∧ t_xmin≠  current_txid ⇒ Invisible



 **Status of t_xmin is COMMITTED**

- t_xmin 상태가 COMMITTED인 튜플은 세 가지 조건을 제외하고 표시 됨 (규칙 6,8,9).



```
 /* t_xmin status == COMMITTED */
        IF t_xmin status is 'COMMITTED' THEN
Rule 5:        IF t_xmin is 'active' in the obtained transaction snapshot THEN
                      RETURN 'Invisible'
Rule 6:        ELSE IF t_xmax = INVALID OR status of t_xmax is 'ABORTED' THEN
                      RETURN 'Visible'
               ELSE IF t_xmax status is 'IN_PROGRESS' THEN
Rule 7:               IF t_xmax =  current_txid THEN
                             RETURN 'Invisible'
Rule 8:               ELSE  /* t_xmax != current_txid */
                             RETURN 'Visible'
                      END IF
               ELSE IF t_xmax status is 'COMMITTED' THEN
Rule 9:               IF t_xmax is 'active' in the obtained transaction snapshot THEN
                             RETURN 'Visible'
Rule 10:              ELSE
                             RETURN 'Invisible'
                      END IF
               END IF
        END IF
```

- t_xmax가 INVALID 또는 ABORTED이기 때문에 규칙 6은 분명.
- 세 가지 예외 조건과 규칙 8 및 9는 다음과 같이 설명됨.

- 첫 번째 예외 조건은 획득한 트랜잭션 스냅샷에서 t_xmin이 활성화 되어 있다는 것(Rule 5).
  - 이 조건에서는 t_xmin이 진행 중인 것으로 처리되어야 하므로 이 튜플은 보이지 않음.

- 두 번째 예외 조건은 t_xmax가 현재 txid라는 것(규칙 7).
  - 이 조건에서는 규칙 3과 마찬가지로 이 튜플은 이 트랜잭션 자체에 의해 업데이트되거나 삭제되었기 때문에 보이지 않음.

- 대조적으로, t_xmax의 상태가 IN_PROGRESS이고 t_xmax가 현재 txid가 아닌 경우(규칙 8) 튜플은 삭제되지 않았기 때문에 표시 됨.

- 세 번째 예외 조건은 t_xmax의 상태가 COMMITTED이고 t_xmax가 획득한 트랜잭션 스냅샷에서 활성 상태 가 아닌 것(규칙 10).
  - 이 조건에서는 이 튜플이 다른 트랜잭션에 의해 업데이트되거나 삭제되었기 때문에 표시되지 않음

- 대조적으로, t_xmax의 상태가 COMMITTED이지만 획득된 트랜잭션 스냅샷에서 t_xmax가 활성 상태인 경우(규칙 9) t_xmax가 진행 중인 것으로 처리되어야 하므로 튜플이 표시 됨.

**규칙 5:**
- Status(t_xmin) = COMMITTED인 경우 ∧ Snapshot(t_xmin) = active  ⇒ Invisible

**규칙 6:**
- Status(t_xmin) = COMMITTED인 경우 ∧ (t_xmax = INVALID ∨ Status(t_xmax) = 중단됨) ⇒ Visible

**규칙 7:**
- Status(t_xmin) = COMMITTED인 경우 ∧ Status(t_xmax) = IN_PROGRESS ∧ t_xmax = current_txid ⇒ Invisible

**규칙 8:**
- Status(t_xmin) = COMMITTED인 경우 ∧ Status(t_xmax) = IN_PROGRESS∧ t_xmax
≠ current_txid ⇒ Visible

**규칙 9:**
- Status(t_xmin) = COMMITTED인 경우 ∧ Status(t_xmax) = COMMITTED∧ Snapshot(t_xmax) = active  ⇒ Visible

**규칙 10:**
- Status(t_xmin) = COMMITTED인 경우 ∧ Status(t_xmax) = COMMITTED∧ Snapshot(t_xmax)
≠ active  ⇒ Invisible